$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n) (\sigma^x_{m} \sigma^x_{n} + \sigma^y_{m} \sigma^y_{n} + \sigma^z_{m} \sigma^z_n) $ where $J_{HS}(m-n) = \frac{\\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [1]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt

In [2]:
def wrap_index(N, m, n):
    if m >= n:
        swap = m
        m = n
        n = swap
    
    forward_dist = abs(m-n)
    back_dist = abs(N+m-n)
    return min(forward_dist, back_dist)

In [3]:
def haldane_shastry_model(N):
    hi = nk.hilbert.Spin(s=1/2, N=N)
    sum = 0
    for n in range(N):
        for m in range(n):
            hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(N, n, m))*np.pi/N)**2))
            x = sigmax(hi, m) * sigmax(hi, n)
            y = sigmay(hi, m) * sigmay(hi, n)
            z = sigmaz(hi, m) * sigmaz(hi, n)
            sum += hs * (x+y+z)
    return sum / 4

In [4]:
nqs_en_x = []
nqs_en_y = []
analytical_en_x = []
analytical_en_y = []

In [5]:
for N in range(3, 31):
    if N % 2 == 0:
        hs = haldane_shastry_model(N)
        rbm = nk.models.RBM(alpha=1, param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))

        hi = nk.hilbert.Spin(s=1/2, N=N)
        sampler = nk.sampler.MetropolisLocal(hi)

        vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)
        optimizer = nk.optimizer.Sgd(learning_rate=0.1)
        preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
        vmc = nk.driver.VMC(hs, optimizer, variational_state=vstate, preconditioner=preconditioner)

        log = nk.logging.RuntimeLog()
        vmc.run(n_iter=250 + 15 * N, out=log)
        opt_energy = vstate.expect(hs)

        nqs_en_x.append(N)
        analytical_en_x.append(N)
        nqs_en_y.append(opt_energy.mean.real)
        analytical_en_y.append(-np.pi**2 / 24 * (N + 5 / N))

100%|██████████| 700/700 [03:34<00:00,  3.27it/s, Energy=-11.595-0.015j ± 0.017 [σ²=0.140, R̂=1.2937]] 


In [11]:
np.savetxt("hs_outputs.txt", np.column_stack((nqs_en_x, nqs_en_y, analytical_en_x, analytical_en_y)))